In [39]:
from alpaca.data.historical import CryptoHistoricalDataClient


# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [40]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from enum import Enum


class TimeFrameUnit(Enum):
    Minute = "Min"
    Hour = "Hour"
    Day = "Day"
    Week = "Week"
    Month = "Month"

timeframe_10min = TimeFrame(amount=10, unit=TimeFrameUnit.Minute)



# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=timeframe_10min,
  start="2018-05-25",
  end="2024-05-26"
)

In [31]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df



open        high         low  \
symbol  timestamp                                                       
BTC/USD 2021-01-01 06:00:00+00:00  29255.7100  29338.2500  29237.4500   
        2021-01-01 06:10:00+00:00  29280.2100  29315.6300  29210.0500   
        2021-01-01 06:20:00+00:00  29212.8200  29285.0500  29173.7600   
        2021-01-01 06:30:00+00:00  29175.7700  29238.8000  29153.5500   
        2021-01-01 06:40:00+00:00  29224.1300  29287.4100  29212.9200   
...                                       ...         ...         ...   
        2024-05-25 23:20:00+00:00  69318.5600  69330.1400  69318.5600   
        2024-05-25 23:30:00+00:00  69330.1005  69330.1005  69274.8550   
        2024-05-25 23:40:00+00:00  69216.2340  69247.5450  69216.2340   
        2024-05-25 23:50:00+00:00  69255.2835  69291.3885  69255.2835   
        2024-05-26 00:00:00+00:00  69244.1030  69244.1030  69244.1030   

                                        close     volume  trade_count  \
symbol  timestamp                                                       
BTC/USD 2021-01-01 06:00:00+00:00  29300.0000   5.121630        251.0   
        2021-01-01 06:10:00+00:00  29212.1600   5.985020        213.0   
        2021-01-01 06:20:00+00:00  29192.6100   7.079464        244.0   
        2021-01-01 06:30:00+00:00  29224.1200  11.705320        221.0   
        2021-01-01 06:40:00+00:00  29235.6500   5.750293        122.0   
...                                       ...        ...          ...   
        2024-05-25 23:20:00+00:00  69330.1400   0.000000          0.0   
        2024-05-25 23:30:00+00:00  69296.0200   0.000000          0.0   
        2024-05-25 23:40:00+00:00  69247.5450   0.000015          1.0   
        2024-05-25 23:50:00+00:00  69291.3885   0.000000          0.0   
        2024-05-26 00:00:00+00:00  69244.1030   0.000000          0.0   

                                           vwap  
symbol  timestamp                                
BTC/USD 2021-01-01 06:00:00+00:00  29294.514194  
        2021-01-01 06:10:00+00:00  29284.921053  
        2021-01-01 06:20:00+00:00  29210.504490  
        2021-01-01 06:30:00+00:00  29214.144326  
        2021-01-01 06:40:00+00:00  29249.214998  
...                                         ...  
        2024-05-25 23:20:00+00:00      0.000000  
        2024-05-25 23:30:00+00:00      0.000000  
        2024-05-25 23:40:00+00:00  69216.234000  
        2024-05-25 23:50:00+00:00      0.000000  
        2024-05-26 00:00:00+00:00      0.000000  

[178438 rows x 7 columns]

In [37]:
df_btc_bars = btc_bars.df

df_btc_bars.to_csv("../src/data/btc_bars.csv")

In [33]:
# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["ETH/USD"],
  timeframe=timeframe_10min,
  start="2018-05-25",
  end="2024-05-26"
)
eth_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
eth_bars.df

open       high        low      close  \
symbol  timestamp                                                               
ETH/USD 2021-01-01 06:00:00+00:00   742.1500   744.6100   742.1500   743.5000   
        2021-01-01 06:10:00+00:00   743.6900   743.9100   742.0000   742.0000   
        2021-01-01 06:20:00+00:00   742.2800   742.2800   738.1600   739.2500   
        2021-01-01 06:30:00+00:00   739.2100   741.4800   738.6400   741.4800   
        2021-01-01 06:40:00+00:00   740.5600   741.4100   739.5300   740.8000   
...                                      ...        ...        ...        ...   
        2024-05-25 23:20:00+00:00  3746.2250  3751.6075  3746.2250  3751.3965   
        2024-05-25 23:30:00+00:00  3749.5405  3749.5405  3745.9025  3745.9025   
        2024-05-25 23:40:00+00:00  3747.4050  3747.4050  3747.4050  3747.4050   
        2024-05-25 23:50:00+00:00  3749.7860  3751.0550  3749.7860  3749.9360   
        2024-05-26 00:00:00+00:00  3747.2350  3748.7165  3746.6400  3746.6400   

                                     volume  trade_count        vwap  
symbol  timestamp                                                     
ETH/USD 2021-01-01 06:00:00+00:00   4.96296         16.0  743.556855  
        2021-01-01 06:10:00+00:00   8.21388         17.0  742.981265  
        2021-01-01 06:20:00+00:00  30.99740         31.0  739.276616  
        2021-01-01 06:30:00+00:00  77.14087         37.0  740.692151  
        2021-01-01 06:40:00+00:00  24.31402         20.0  741.078805  
...                                     ...          ...         ...  
        2024-05-25 23:20:00+00:00   0.00000          0.0    0.000000  
        2024-05-25 23:30:00+00:00   0.00000          0.0    0.000000  
        2024-05-25 23:40:00+00:00   0.00000          0.0    0.000000  
        2024-05-25 23:50:00+00:00   0.00000          0.0    0.000000  
        2024-05-26 00:00:00+00:00   0.00000          0.0    0.000000  

[178453 rows x 7 columns]

In [38]:
df_eth_bars = eth_bars.df

df_eth_bars.to_csv("../src/data/eth_bars.csv")